# Production System Simulation
## Modeling a Manufacturing Facility with Three Product Families

This notebook simulates a production system with:
- 6 working centers (each with capacity 1)
- 3 product families with different routing probabilities
- Gamma-distributed processing times
- Due dates uniformly distributed between 30-50 time units

Key performance indicators calculated:
- Work in Progress (WIP)
- Tardiness and Earliness (max and average)
- Throughput
- Working center utilization
- Job arrivals per family over time

# Todos

aggrega le funzioni per la simulazione con stampe di debug e senza.

Calcola gli intervalli di confidenza in modo sequenziale.

In [ ]:
# Install required packages
%pip install simpy numpy matplotlib scipy

In [ ]:
import simpy
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import gamma
plt.style.use('ggplot')

## Simulation Parameters

In [ ]:
# Simulation configuration
SIM_TIME = 10_000  # Total simulation time
WARM_UP = 3_000    # Warm-up period to ignore in statistics (determined with Welch's method)

# Working center IDs
WC_IDS = [1, 2, 3, 4, 5, 6]

# Product family probabilities [F1, F2, F3]
FAMILY_PROBS = [0.10, 0.52, 0.38]

# Gamma distribution parameters (alpha, beta) for each family
PROCESSING_PARAMS = {
    1: (2, 2),      # Family 1
    2: (4, 0.5),    # Family 2
    3: (6, 1/6)     # Family 3
}

# Routing logic: (WC, probability) for each family
ROUTING = {
    1: [(1, 1.0), (2, 1.0), (4, 1.0), (5, 1.0), (6, 1.0)],
    2: [(1, 0.8), (2, 0.8), (3, 1.0), (4, 0.8), (5, 0.8), (6, 0.75)],
    3: [(3, 1.0), (6, 0.75)]
}

# Arrival rate (exponential distribution)
ARRIVAL_RATE = 0.65  # lambda

## System Visualization

In [ ]:
def visualize_processing_distributions():
    """Visualize gamma distributions for each product family"""
    plt.figure(figsize=(10, 6))
    
    # Generate x values
    x = np.linspace(0, 20, 1000)
    
    # Plot each family's distribution
    for family, (alpha, beta) in PROCESSING_PARAMS.items():
        y = gamma.pdf(x, alpha, scale=beta)
        plt.plot(x, y, label=f'Family {family} (α={alpha}, β={beta:.2f})', linewidth=2.5)
    
    plt.title('Processing Time Distributions by Product Family')
    plt.xlabel('Processing Time')
    plt.ylabel('Probability Density')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

def visualize_routing_probabilities():
    """Visualize routing probabilities for each product family"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    wc_positions = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}
    
    for idx, (family, routing) in enumerate(ROUTING.items()):
        ax = axes[idx]
        wcs = [wc for wc, _ in routing]
        probs = [prob for _, prob in routing]
        
        # Create positions
        positions = [wc_positions[wc] for wc in wcs]
        
        ax.bar(positions, probs, color=plt.cm.tab10(family-1), alpha=0.7)
        ax.set_title(f'Family {family} Routing Probabilities')
        ax.set_xlabel('Working Center')
        ax.set_ylabel('Probability')
        ax.set_xticks(list(wc_positions.values()))
        ax.set_xticklabels(list(wc_positions.keys()))
        ax.set_ylim(0, 1.1)
        
        # Add value labels
        for i, p in enumerate(probs):
            ax.text(positions[i], p + 0.02, f'{p:.2f}', ha='center')
    
    plt.tight_layout()
    plt.show()

# Visualize system characteristics
visualize_processing_distributions()
visualize_routing_probabilities()

## Simulation Implementation

In [ ]:
class Job:
    """Represents a production job with family, due date, and tracking times"""
    next_id = 1
    
    def __init__(self, env, family):
        self.jid = Job.next_id
        Job.next_id += 1
        self.family = family
        self.arrival_time = env.now
        self.due_date = self.arrival_time + random.uniform(30, 50)
        self.start_times = {}
        self.finish_times = {}
        self.completion_time: float | None = None
        
    def __repr__(self):
        return f"Job {self.jid} (F{self.family}) due @ {self.due_date:.2f}"

In [ ]:
def job_generator(env, wcs, stats, debug: bool = False):
    """Generates jobs with exponential inter-arrival times"""
    while True:
        family = random.choices([1, 2, 3], weights=FAMILY_PROBS)[0]
        job = Job(env, family)
        stats['generated'] += 1
        stats['wip_count'] += 1
        stats['wip_history'].append((env.now, stats['wip_count']))
        
        # Record family-specific statistics
        stats['family_counts'][family] += 1
        
        # Record arrival time for this family
        stats['arrival_times'][family].append(env.now)

        if debug:
            print(f"\n[🆕 ARRIVAL @ {env.now:.2f}] {job} ")
        
        # Start processing this job
        env.process(job_process(env, job, wcs, stats, debug))
        
        # Wait for next arrival
        interarrival = random.expovariate(ARRIVAL_RATE)
        yield env.timeout(interarrival)

In [ ]:
def job_process(env : simpy.Environment, job : Job, wcs : dict[int, simpy.Resource], stats, debug: bool = False):
    """Processes a job through its routing sequence"""
    for wc_id, prob in ROUTING[job.family]:
        if random.random() < prob:  # Decide whether to visit this WC
            wc = wcs[wc_id]
            with wc.request() as req:
                if debug:
                    print(f"[⏳ QUEUE @ {env.now:.2f}] {job} "
                          f"waiting for WC {wc_id}")
                # Record queue start time
                queue_start = env.now
                
                # Request the working center
                yield req
                
                # Record queue time
                queue_time = env.now - queue_start
                stats['queue_times'][wc_id].append(queue_time)
                
                # Record processing start
                job.start_times[wc_id] = env.now
                
                # Get processing time
                alpha, beta = PROCESSING_PARAMS[job.family]
                processing_time = random.gammavariate(alpha, beta)

                if debug:
                    print(f"[⚙️ START @ {env.now:.2f}] {job} "
                          f"processing at WC {wc_id} for {processing_time:.2f} units")
                
                # Process the job
                yield env.timeout(processing_time)
                
                # Record processing finish
                job.finish_times[wc_id] = env.now
                
                # Update statistics
                stats['wc_busy_time'][wc_id] += processing_time
                stats['wc_utilization'][wc_id].append((env.now, stats['wc_busy_time'][wc_id] / env.now))

                if debug:
                    print(f"[✅ FINISH @ {env.now:.2f}] {job} "
                          f"completed at WC {wc_id}")
    
    # Job completion
    job.completion_time = env.now
    stats['completed_jobs'].append(job)
    stats['wip_count'] -= 1
    stats['wip_history'].append((env.now, stats['wip_count']))

    # Calculate and record tardiness/earliness
    tardiness = max(0, job.completion_time - job.due_date)
    earliness = max(0, job.due_date - job.completion_time)

    if debug:
        print(f"\n[🏁 COMPLETE @ {env.now:.2f}] {job} "
              f"Tardiness: {tardiness:.2f} | Earliness: {earliness:.2f}")
    
    # Only record statistics after warm-up period
    if env.now > WARM_UP:
        stats['tardiness'].append(tardiness)
        stats['earliness'].append(earliness)
        stats['family_tardiness'][job.family].append(tardiness)
        stats['family_earliness'][job.family].append(earliness)
        
        # Update throughput counter
        stats['completed_after_warmup'] += 1
        
        # Update throughput immediately after completion
        elapsed_time = env.now - WARM_UP
        current_throughput = stats['completed_after_warmup'] / elapsed_time
        stats['throughput_over_time'].append((env.now, current_throughput))
        stats['last_throughput_sample_time'] = env.now

In [ ]:
def run_simulation(simulation_time: float = SIM_TIME, debug: bool = False):
    """Run the simulation and return statistics"""
    if debug:
        print("="*80)
        print(f"DEBUG SIMULATION RUN ({simulation_time} TIME UNITS)")
        print("="*80)
    
    # Initialize environment
    env = simpy.Environment()
    
    # Initialize statistics collection
    stats = {
        'generated': 0,
        'completed_jobs': [],
        'completed_after_warmup': 0,
        'wip_count': 0,
        'wip_history': [],
        'wc_busy_time': {wc: 0 for wc in WC_IDS},
        'wc_utilization': {wc: [] for wc in WC_IDS},
        'queue_times': {wc: [] for wc in WC_IDS},
        'tardiness': [],
        'earliness': [],
        'family_counts': {1: 0, 2: 0, 3: 0},
        'family_tardiness': {1: [], 2: [], 3: []},
        'family_earliness': {1: [], 2: [], 3: []},
        'throughput_over_time': [],  # (time, current_throughput)
        'last_throughput_sample_time': WARM_UP,
        'arrival_times': {1: [], 2: [], 3: []}  # Track arrival times per family
    }
    
    # Create working centers (each as a Resource with capacity 1)
    wcs = {wc_id: simpy.Resource(env) for wc_id in WC_IDS}
    
    # Start processes
    env.process(job_generator(env, wcs, stats, debug))
    
    # Run simulation
    env.run(until=simulation_time)

    if debug:
        print("\n" + "="*80)
        print("DEBUG SIMULATION SUMMARY")
        print("="*80)
        print(f"Total jobs generated: {stats['generated']}")
        print(f"Jobs completed: {len(stats['completed_jobs'])}")
        print(f"Current WIP: {stats['wip_count']}")
        print(f"Family distribution: F1={stats['family_counts'][1]}, F2={stats['family_counts'][2]}, F3={stats['family_counts'][3]}")
        print("\nWorking Center Utilization:")
        for wc in WC_IDS:
            utilization = stats['wc_busy_time'][wc] / simulation_time
            print(f"  WC {wc}: {utilization:.2%}")
    
    return stats

# Model verification

Run a small simulation with useful prints

In [ ]:
DEBUG_SIMULATION_TIME = 20
random.seed(42)

_ = run_simulation(simulation_time=DEBUG_SIMULATION_TIME, debug=True)

## Running the Simulation

In [ ]:
random.seed(42)

# Run the simulation
print("Starting simulation...")
stats = run_simulation()
print("Simulation completed!")

# Filter out warm-up period jobs
completed_after_warmup = [job for job in stats['completed_jobs'] if job.arrival_time > WARM_UP]
num_completed = len(completed_after_warmup)

# Calculate KPIs
if num_completed > 0:
    # Tardiness and earliness
    avg_tardiness = np.mean(stats['tardiness']) if stats['tardiness'] else 0
    max_tardiness = np.max(stats['tardiness']) if stats['tardiness'] else 0
    
    avg_earliness = np.mean(stats['earliness']) if stats['earliness'] else 0
    max_earliness = np.max(stats['earliness']) if stats['earliness'] else 0
    
    # Throughput
    throughput = num_completed / (SIM_TIME - WARM_UP)
    
    # WIP calculations
    wip_history = np.array(stats['wip_history'])
    wip_after_warmup = wip_history[wip_history[:,0] > WARM_UP]
    avg_wip = np.mean(wip_after_warmup[:,1]) if len(wip_after_warmup) > 0 else 0
    
    # Utilization
    utilizations = {wc: stats['wc_busy_time'][wc] / SIM_TIME for wc in WC_IDS}
    
    # Queue times
    avg_queue_times = {wc: np.mean(stats['queue_times'][wc]) if stats['queue_times'][wc] else 0 
                      for wc in WC_IDS}
    
    # Family-specific tardiness
    family_avg_tardiness = {
        fam: np.mean(tardiness) if tardiness else 0
        for fam, tardiness in stats['family_tardiness'].items()
    }
    
    # Compile KPIs
    kpis = {
        'Average WIP': avg_wip,
        'Maximum Tardiness': max_tardiness,
        'Average Tardiness': avg_tardiness,
        'Maximum Earliness': max_earliness,
        'Average Earliness': avg_earliness,
        'Throughput (jobs/time unit)': throughput,
        'Working Center Utilization': utilizations,
        'Average Queue Times': avg_queue_times,
        'Family Average Tardiness': family_avg_tardiness
    }
else:
    kpis = {}

# Print summary statistics
print(f"\n{' Summary Statistics ':-^60}")
print(f"Total jobs generated: {stats['generated']}")
print(f"Jobs completed after warm-up: {num_completed}")
print(f"Family distribution: F1={stats['family_counts'][1]}, F2={stats['family_counts'][2]}, F3={stats['family_counts'][3]}")

print(f"\n{' Key Performance Indicators ':-^60}")
for k, v in kpis.items():
    if isinstance(v, dict):
        print(f"\n{k}:")
        for subk, subv in v.items():
            print(f"  {subk}: {subv:.4f}")
    else:
        print(f"{k}: {v:.4f}")

# Model validation

Verify these results against Little's law.

In [ ]:
def verify_littles_law(stats, kpis):
    """Verify simulation results using Little's Law"""
    print("\n" + "="*80)
    print("ANALYTICAL VERIFICATION USING LITTLE'S LAW")
    print("="*80)
    
    # Get relevant statistics
    lambda_ = ARRIVAL_RATE  # Arrival rate (jobs/time unit)
    avg_wip = kpis['Average WIP']
    throughput = kpis['Throughput (jobs/time unit)']
    
    # Calculate average flow time from completed jobs
    flow_times = [
                    job.completion_time - job.arrival_time 
                    for job in stats['completed_jobs'] 
                    if job.arrival_time > WARM_UP
                ]
    avg_flow_time = np.mean(flow_times) if flow_times else 0
    
    # Little's Law: L = λ * W
    # Where:
    #   L = average number of items in the system (WIP)
    #   λ = long-term average arrival rate
    #   W = average time an item spends in the system (flow time)
    
    # Version 1: Using arrival rate
    calculated_wip = lambda_ * avg_flow_time
    
    # Version 2: Using throughput (more stable)
    calculated_wip_alt = throughput * avg_flow_time
    
    # Calculate flow time from WIP and throughput
    calculated_flow_time = avg_wip / throughput
    
    # Print results
    print(f"Input Arrival Rate (λ): {lambda_:.4f} jobs/time unit")
    print(f"Measured Throughput: {throughput:.4f} jobs/time unit")
    print(f"Average Flow Time (W): {avg_flow_time:.4f} time units")
    print(f"Average WIP (L) from simulation: {avg_wip:.4f}")
    print(f"Calculated WIP (λ * W): {calculated_wip:.4f}")
    print(f"Calculated WIP (Throughput * W): {calculated_wip_alt:.4f}")
    print(f"Calculated Flow Time (L / Throughput): {calculated_flow_time:.4f}")
    
    # Calculate differences
    diff_percent = abs(avg_wip - calculated_wip) / avg_wip * 100 if avg_wip > 0 else 0
    diff_percent_alt = abs(avg_wip - calculated_wip_alt) / avg_wip * 100 if avg_wip > 0 else 0
    flow_diff_percent = abs(avg_flow_time - calculated_flow_time) / avg_flow_time * 100 if avg_flow_time > 0 else 0
    
    print("\n" + "-"*80)
    print(f"Consistency Check Results:")
    print(f"  WIP Difference (λ*W vs simulation): {diff_percent:.2f}%")
    print(f"  WIP Difference (Throughput*W vs simulation): {diff_percent_alt:.2f}%")
    print(f"  Flow Time Difference: {flow_diff_percent:.2f}%")
    
    # Interpretation
    print("\n" + "-"*80)
    print("Interpretation:")
    if diff_percent < 5 and diff_percent_alt < 5 and flow_diff_percent < 5:
        print("✅ Excellent consistency with Little's law - Differences < 5%")
    elif diff_percent < 10 and diff_percent_alt < 10 and flow_diff_percent < 10:
        print("⚠️ Good consistency with Little's law - Differences < 10%")
    else:
        print("❌ Significant discrepancies with Little's law detected")

# Run verification if we have valid KPIs
if kpis and 'Average WIP' in kpis and 'Throughput (jobs/time unit)' in kpis:
    verify_littles_law(stats, kpis)
else:
    print("Skipping Little's Law verification - insufficient data")

In [ ]:
def wc_utilization_validation(stats, kpis):
    """Validate working center utilization analytically"""
    print("\n" + "="*80)
    print("WORKING CENTER UTILIZATION VALIDATION")
    print("="*80)
    print("This validation compares simulated utilization with theoretical expectations\n")
    
    # Configuration parameters
    family_probs = {1: 0.10, 2: 0.52, 3: 0.38}
    gamma_means = {fam: alpha*beta for fam, (alpha, beta) in PROCESSING_PARAMS.items()}
    
    # Define routing probabilities per family per WC
    routing_probs = {
        1: {1: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 6: 1.0},
        2: {1: 0.8, 2: 0.8, 3: 1.0, 4: 0.8, 5: 0.8, 6: 0.75},
        3: {3: 1.0, 6: 0.75}
    }
    
    # Calculate arrival rate to each WC
    wc_arrival_rates = {wc: 0.0 for wc in WC_IDS}
    for family in [1, 2, 3]:
        for wc, prob in routing_probs[family].items():
            wc_arrival_rates[wc] += ARRIVAL_RATE * family_probs[family] * prob
    
    # Calculate expected utilization for each WC
    util_results = []
    for wc in WC_IDS:
        arrival_rate = wc_arrival_rates[wc]
        
        # Calculate weighted average processing time for this WC
        processing_times = []
        weights = []
        for family in [1, 2, 3]:
            if wc in routing_probs[family]:
                weight = ARRIVAL_RATE * family_probs[family] * routing_probs[family][wc]
                weights.append(weight)
                processing_times.append(gamma_means[family])
        
        if weights:
            # Calculate weighted average processing time
            total_weight = sum(weights)
            weights = [w/total_weight for w in weights]  # Normalize weights
            avg_processing_time = sum(p * w for p, w in zip(processing_times, weights))
            
            # Utilization = Arrival Rate × Average Processing Time
            expected_util = arrival_rate * avg_processing_time
            simulated_util = kpis['Working Center Utilization'][wc]
            diff = abs(expected_util - simulated_util)
            diff_percent = diff / simulated_util * 100 if simulated_util > 0 else 0
            
            util_results.append({
                'WC': wc,
                'Expected': expected_util,
                'Simulated': simulated_util,
                'Difference (%)': diff_percent
            })
    
    # Print results
    print("{:<5} {:<12} {:<12} {:<12}".format('WC', 'Expected', 'Simulated', 'Difference (%)'))
    print("-" * 45)
    for res in util_results:
        print("{:<5} {:<12.4f} {:<12.4f} {:<12.2f}%".format(
            f"WC {res['WC']}", 
            res['Expected'], 
            res['Simulated'], 
            res['Difference (%)']
        ))
    
    # Interpretation
    print("\nValidation Summary:")
    print(f"- Average difference: {np.mean([res['Difference (%)'] for res in util_results]):.2f}%")
    print(f"- Maximum difference: {np.max([res['Difference (%)'] for res in util_results]):.2f}%")
    
    if all(res['Difference (%)'] < 10 for res in util_results):
        print("✅ Validation PASSED - All utilizations within 10% of theoretical values")
    elif all(res['Difference (%)'] < 15 for res in util_results):
        print("⚠️ Validation ACCEPTABLE - All utilizations within 15% of theoretical values")
    else:
        print("❌ Validation FAILED - Some utilizations differ significantly from theoretical values")

# Run WC utilization validation
if kpis and 'Working Center Utilization' in kpis:
    wc_utilization_validation(stats, kpis)
else:
    print("Skipping WC utilization validation - insufficient data")

In [ ]:
def throughput_validation(stats, kpis):
    """Validate throughput analytically"""
    print("\n" + "="*80)
    print("THROUGHPUT VALIDATION")
    print("="*80)
    print("This validation compares simulated throughput with theoretical expectations\n")
    
    # Theoretical throughput should equal arrival rate in stable systems
    expected_throughput = ARRIVAL_RATE
    simulated_throughput = kpis['Throughput (jobs/time unit)']
    
    # Calculate difference
    diff = abs(expected_throughput - simulated_throughput)
    diff_percent = diff / expected_throughput * 100
    
    # Print results
    print("{:<12} {:<12} {:<12}".format('Metric', 'Expected', 'Simulated'))
    print("-" * 35)
    print("{:<12} {:<12.4f} {:<12.4f}".format(
        'Throughput', 
        expected_throughput, 
        simulated_throughput
    ))
    print("\n{:<12} {:<12.2f}%".format('Difference', diff_percent))
    
    # Interpretation
    print("\nValidation Summary:")
    if diff_percent < 2:
        print("✅ Excellent match between thoretical and expected throughput - Difference < 2%")
    elif diff_percent < 5:
        print("⚠️ Good match between thoretical and expected throughput - Difference < 5%")
    elif diff_percent < 10:
        print("⚠️ Acceptable match between thoretical and expected throughput - Difference < 10%")
    else:
        print("❌ Significant discrepancy between thoretical and expected throughput - Difference ≥ 10%")

# Run throughput validation
if kpis and 'Throughput (jobs/time unit)' in kpis:
    throughput_validation(stats, kpis)
else:
    print("Skipping throughput validation - insufficient data")

## Results Visualization

In [ ]:
def visualize_results(stats, kpis):
    """Create visualizations of simulation results"""
    # Create figure layout
    plt.figure(figsize=(15, 15))
    
    # WIP over time
    plt.subplot(3, 2, 1)
    wip_time, wip_count = zip(*stats['wip_history'])
    plt.plot(wip_time, wip_count, color='royalblue')
    plt.axvline(WARM_UP, color='r', linestyle='--', label='Warm-up end')
    plt.title('Work in Progress (WIP) Over Time')
    plt.xlabel('Simulation Time')
    plt.ylabel('WIP Count')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Throughput over time (updated per completion)
    plt.subplot(3, 2, 2)
    if stats['throughput_over_time']:
        t_time, throughput = zip(*stats['throughput_over_time'])
        plt.plot(t_time, throughput, 'g-', alpha=0.7)
        plt.axhline(kpis['Throughput (jobs/time unit)'], color='r', linestyle='--', 
                   label=f'Avg: {kpis["Throughput (jobs/time unit)"]:.4f}')
        plt.axvline(WARM_UP, color='r', linestyle='--')
        plt.title('Throughput Evolution (Per Job Completion)')
        plt.xlabel('Simulation Time')
        plt.ylabel('Throughput (jobs/time unit)')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # Utilization over time
    plt.subplot(3, 2, 3)
    for wc in WC_IDS:
        if stats['wc_utilization'][wc]:
            util_time, util = zip(*stats['wc_utilization'][wc])
            plt.plot(util_time, util, label=f'WC {wc}')
    plt.axvline(WARM_UP, color='r', linestyle='--')
    plt.title('Working Center Utilization Over Time')
    plt.xlabel('Simulation Time')
    plt.ylabel('Utilization')
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Tardiness distribution (only positive values)
    plt.subplot(3, 2, 4)
    if stats['tardiness']:
        # Filter out zero tardiness values
        positive_tardiness = [t for t in stats['tardiness'] if t > 0]
        
        if positive_tardiness:
            plt.hist(positive_tardiness, bins=50, color='salmon', alpha=0.7)
            plt.axvline(kpis['Average Tardiness'], color='darkred', 
                       linestyle='--', label=f'Avg: {kpis["Average Tardiness"]:.2f}')
            plt.title('Job Tardiness Distribution (Positive Values Only)')
            plt.xlabel('Tardiness')
            plt.ylabel('Frequency')
            plt.legend()
            plt.grid(True, alpha=0.3)
    
    # Earliness distribution (only positive values)
    plt.subplot(3, 2, 5)
    if stats['earliness']:
        # Filter out zero earliness values
        positive_earliness = [e for e in stats['earliness'] if e > 0]
        
        if positive_earliness:
            plt.hist(positive_earliness, bins=50, color='lightgreen', alpha=0.7)
            plt.axvline(kpis['Average Earliness'], color='darkgreen', 
                       linestyle='--', label=f'Avg: {kpis["Average Earliness"]:.2f}')
            plt.title('Job Earliness Distribution (Positive Values Only)')
            plt.xlabel('Earliness')
            plt.ylabel('Frequency')
            plt.legend()
            plt.grid(True, alpha=0.3)
    
    # Utilization bar chart with 85% target
    plt.subplot(3, 2, 6)
    wcs = list(kpis['Working Center Utilization'].keys())
    utils = [kpis['Working Center Utilization'][wc] for wc in wcs]
    plt.bar([f'WC {wc}' for wc in wcs], utils, color='teal', alpha=0.7)
    
    # 85% target line explanation
    plt.axhline(0.85, color='r', linestyle='--', alpha=0.5, 
               label='85% Target (Optimal Utilization)')
    plt.title('Working Center Utilization')
    plt.xlabel('Working Center')
    plt.ylabel('Utilization')
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add value labels
    for i, v in enumerate(utils):
        plt.text(i, v + 0.02, f"{v:.2%}", ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Family-specific performance
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Family tardiness
    family_tardiness = [
        stats['family_tardiness'][1],
        stats['family_tardiness'][2],
        stats['family_tardiness'][3]
    ]
    axes[0].boxplot(family_tardiness, 
                   tick_labels=['Family 1', 'Family 2', 'Family 3'])
    axes[0].set_title('Tardiness by Product Family')
    axes[0].set_ylabel('Tardiness')
    
    # Family earliness
    family_earliness = [
        stats['family_earliness'][1],
        stats['family_earliness'][2],
        stats['family_earliness'][3]
    ]
    axes[1].boxplot(family_earliness, 
                   tick_labels=['Family 1', 'Family 2', 'Family 3'])
    axes[1].set_title('Earliness by Product Family')
    axes[1].set_ylabel('Earliness')
    
    plt.tight_layout()
    plt.show()
    
    # Job arrivals per family over time
    plt.figure(figsize=(12, 6))
    
    # Cumulative arrivals
    plt.subplot(1, 2, 1)
    for family in [1, 2, 3]:
        arrival_times = sorted(stats['arrival_times'][family])
        cumulative_counts = np.arange(1, len(arrival_times) + 1)
        plt.step(arrival_times, cumulative_counts, where='post', 
                label=f'Family {family}', linewidth=2)
    
    plt.axvline(WARM_UP, color='r', linestyle='--', label='Warm-up end')
    plt.title('Cumulative Job Arrivals per Family')
    plt.xlabel('Simulation Time')
    plt.ylabel('Cumulative Arrivals')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Arrival events timeline
    plt.subplot(1, 2, 2)
    colors = {1: 'red', 2: 'green', 3: 'blue'}
    for family in [1, 2, 3]:
        arrival_times = stats['arrival_times'][family]
        plt.scatter(arrival_times, [family]*len(arrival_times), 
                   color=colors[family], alpha=0.3, label=f'Family {family}')
    
    plt.axvline(WARM_UP, color='r', linestyle='--', label='Warm-up end')
    plt.title('Job Arrivals Timeline per Family')
    plt.xlabel('Simulation Time')
    plt.ylabel('Product Family')
    plt.yticks([1, 2, 3], ['Family 1', 'Family 2', 'Family 3'])
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Visualize results
if kpis:
    visualize_results(stats, kpis)